In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Steps I would be following

1. Understand what I need to do
2. Load and visualize the data
3. EDA on the dataset
3. Choose the architecture (do the research)
5. Train and validate the achitecture
6. Test the architecture

### What I need to do?

1. Accurately classify an image into different diseased category or a healthy leaf
2. Accurately distinguish between many diseases, sometimes more than one on a leaf
3. Deal with the rare classes and novel symptoms 
4. Address depth perception, angle, light, shade, physiological age of the leaf
5. ncorporate expert knowledge in identification, annotation, quantification, and guiding computer vision to search for relevant features during learning.

In [ ]:
#import the depedencies

import time
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.image import imread

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from collections import OrderedDict

import cv2 as cv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

### Load the CSV files

In [ ]:
root = '/kaggle/input/plant-pathology-2020-fgvc7/'
train = pd.read_csv(os.path.join(root,'train.csv'))
test = pd.read_csv(os.path.join(root,'test.csv'))
submission = pd.read_csv(os.path.join(root,'sample_submission.csv'))
images = os.path.join(root,'images')

## Visualize the Data

In [ ]:
def show_images(images,typ):
    fig = plt.figure(figsize=(16,4))
    num = len(images)
    for i in range(num):
        a = fig.add_subplot(1,num,i+1)
        a.set_title(typ,fontsize=10)
        image = imread(os.path.join(root,'images',images[i]))
        plt.imshow(image)
        plt.axis('off')
        
col = ['healthy','multiple_diseases','rust','scab']
print(col)

for column in col:
    images = (train[train[column].apply(lambda x: x==1)]['image_id'].sample(4).values)+'.jpg'
    show_images(images,column)

### Loading the data into the dataloader

In [ ]:
def get_path(image):
    return os.path.join(root,'images',str(image)+'.jpg')

train_data = train.copy()
train_data['image_path'] = train_data['image_id'].apply(get_path)
train_labels = train.loc[:,'healthy':'scab']

test_data = test.copy()
test_data['image_path'] = test_data['image_id'].apply(get_path)
test_paths = test_data['image_path']

train_paths,valid_paths,train_labels,valid_labels = train_test_split(train_data['image_path'],train_labels,test_size=0.2,random_state=23,stratify=train_labels)

train_paths.reset_index(drop=True,inplace=True)
train_labels.reset_index(drop=True,inplace=True)
valid_paths.reset_index(drop=True,inplace=True)
valid_labels.reset_index(drop=True,inplace=True)

In [ ]:
#transformations on the image dataset
mytransform = {
    'train': A.Compose([
        A.RandomResizedCrop(height=256,width=256,p=1.0),
        A.Flip(),
        A.ShiftScaleRotate(rotate_limit=1.0,p=0.8),
        A.Normalize(p=1.0),
        ToTensorV2(p=1.0),
    ]),
    'validation': A.Compose([
        A.RandomResizedCrop(height=256,width=256,p=1.0),
        A.Normalize(p=1.0),
        ToTensorV2(p=1.0),
    ])
    
}

#custom dataset loader
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self,image_path,labels=None,test=False,transform=None):
        super().__init__()
        self.image_path = image_path
        self.test = test
        if self.test == False:
            self.labels = labels
            
        self.images_transforms = transform
        
    def __getitem__(self,index):
        if self.test == False:
            labels = self.labels.loc[index,['healthy', 'multiple_diseases', 'rust', 'scab']].values
            labels = torch.from_numpy(labels.astype(np.uint8))
            labels = labels.unsqueeze(-1)
            
        image = cv.imread(self.image_path[index])
        image = cv.cvtColor(image,cv.COLOR_BGR2RGB)
        image_transformed = self.images_transforms(image=image)
            
        if self.test ==False:
            return image_transformed['image'], labels
            
        return image_transformed['image']
        
    def __len__(self):
        return self.image_path.shape[0]

In [ ]:
BATCH_SIZE = 32
epochs = 20 
TRAIN_SIZE = train_labels.shape[0]
VALID_SIZE = valid_labels.shape[0]
learning_rate = 5e-5

### Load the dataset into the dataloaders

In [ ]:

train_images = ImageDataset(image_path=train_paths, labels=train_labels, transform=mytransform["train"])
train_loader = torch.utils.data.DataLoader(train_images, shuffle=True, batch_size = BATCH_SIZE)

valid_images = ImageDataset(image_path=valid_paths, labels=valid_labels, transform=mytransform["validation"])
valid_loader = torch.utils.data.DataLoader(valid_images, shuffle=False, batch_size = BATCH_SIZE)

In [ ]:
images, labels =next(iter(train_loader))
print(images[1].shape)

## Import the model

In [ ]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.require_grad = False
    


In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(nn.Linear(num_ftrs,512),
                        nn.ReLU(),
                        nn.Dropout(p=0.3),
                        nn.Linear(512,4))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class DenseCrossEntropy(nn.Module):
    
    def __init__(self):
        super().__init__()
        
    def forward(self,logits,labels):
        logits = logits.float()
        labels = labels.float()
        
        logprobs = F.log_softmax(logits,dim=1)
        
        loss =-labels*logprobs
        loss = loss.sum(-1)
        
        return loss.mean()

## Training and Validating 

In [ ]:
from torch import optim
criterion = DenseCrossEntropy()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
num_classes = torch.tensor([0,1,2,3])
from sklearn.metrics import roc_auc_score

training_loss = []
validation_loss = []
epochs = 20
model.to(device)
min_val_loss = np.Inf
to_break = 0
for e in range(epochs):
    model.train()
    train_loss = 0
    for images,labels in train_loader:
        images,labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = model(images)
        

        loss = criterion(output,labels.squeeze(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*labels.shape[0]
        
    model.eval()
    valid_loss = 0
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss = criterion(output,labels.squeeze(-1))
        valid_loss += loss.item()*labels.shape[0]
        
    print(f'{e+1}/{epochs}... Training Loss {train_loss/len(train_loader)}.... Validation Loss {valid_loss/len(valid_loader)}')
    if min_val_loss >= valid_loss:
        min_val_loss = valid_loss
        torch.save(model.state_dict(),'checkpoiny.pth')
        
        

## Testing on the validation set

In [ ]:
from sklearn.metrics import roc_auc_score
model.to('cpu')
score = []
model.eval()
for images, labels in valid_loader:
    output = model(images)
    preds = torch.softmax(output,dim=1).data
    preds = preds.numpy()
    labels = labels.numpy()
    try:
        val= roc_auc_score(labels.squeeze(-1),preds,average='macro')
        score.append(val)
        print(val)
    except:
        pass
    
print('roc score: ',sum(score)/len(score))

In [ ]:
test_images = ImageDataset(image_path=test_paths,test=True, transform=mytransform["validation"])
test_loader = torch.utils.data.DataLoader(test_images, shuffle=False, batch_size = BATCH_SIZE)

## Submission

In [ ]:
test_preds =None
submission_df = pd.read_csv(os.path.join(root,"sample_submission.csv"))
model.eval()
model.to(device)
for  images in test_loader:
    images = images.to(device)
    with torch.no_grad():
        output = model(images)
        
        if test_preds is None:
            test_preds = output.data.cpu()
        
        else:
            test_preds = torch.cat((test_preds,output.data.cpu()),dim=0)

submission_df[['healthy','multiple_diseases','rust','scab']] = torch.softmax(test_preds,dim=1)
submission_df.to_csv('submission.csv',index=False)

In [ ]:
submission_df.head()

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

### This notebook was able to achieve the score of 0.94167